In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

import os
slice_name="ndn-test_" + os.getenv('NB_USER')
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

You don't have a slice named ndn-test_sjzhu yet.
Continue to the next step to make one.


We'll find a site that has the appropriate number of cores available (4 routers, 3 producers, 3 consumers for a total of 20 cores) and the appropriate number of NICs (8 for the routers, 6 for the producers and 6 for the consumers for a total of 20 NICs).

In [2]:
exp_requires = {'core': 22*2, 'nic': 22}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break

fablib.show_site(site_name)

Name,PSC
State,Active
Address,"4350 Northern Pike,Monroeville, PA 15146"
Location,"(40.43394339243079, -79.75279924982625)"
Hosts,3
CPUs,6
Cores Available,384
Cores Capacity,384
Cores Allocated,0
RAM Available,1506
RAM Capacity,1506


'<pandas.io.formats.style.Styler object at 0x7fd278e6e2f0>'

In [3]:
# this cell sets up the hosts and router
router_names = ["router1", "router2", "router3", "router4"]
producer_names = ["producer1", "producer2", "producer3"]
consumer_names = ["consumer1", "consumer2", "consumer3"]
node_names = router_names + producer_names + consumer_names
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=2, ram=4, disk=10, image='default_ubuntu_20')

In [4]:
# this cell sets up the links between the routers
index = 1
for r in router_names:
    if index >= len(router_names):
        index = 0
    network_name = "net-" + r + "-" + router_names[index]
    nodes = [r, router_names[index]]
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=network_name).get_interfaces()[0] for node in nodes]
    slice.add_l2network(name=network_name, type="L2Bridge", interfaces = ifaces)
    index += 1

In [5]:
# this cell sets up the links between the routers and the other nodes
nets = [
    {"name": "net-router1-consumer1",   "nodes": ["router1", "consumer1"]},
    {"name": "net-router1-consumer2",  "nodes": ["router1", "consumer2"]},
    {"name": "net-router2-producer1",   "nodes": ["router2", "producer1"]},
    {"name": "net-router3-producer2",  "nodes": ["router3", "producer2"]},
    {"name": "net-router4-producer3",   "nodes": ["router4", "producer3"]},
    {"name": "net-router4-consumer3",  "nodes": ["router4", "consumer3"]},
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)